# Environmental Justice

The challenge requires us to link social and environmental justice.
I found myelf wondering about possible interactions between these 2 fields. 

Fortunately the city of Berlin has responded to the only question in the cdp report directly targeted at the social impact of climate actions: 

> Has your city measured and demonstrated the wider social and economic impacts of delivering climate actions/projects/policies? If so, please provide more details and a link to more information.

Unlike other cities Berlin has already put measures into place and link their detailed plan of measureing Environmental Justice in Berlin and how they couple this to investments.

The Dataset consists of 5 Main indicators for Environmental Justice summarized to the easy understandable rating of very low, low, middle and high fully based on objective parameters for the 447 planning areas of Berlin:
    1. Noise
        (noise costs (the harm caused per person by the impact of the disturbance and the health risks as a sum) per inhabitant)
    2. Air Pollution
        (measured using PM2.5 + NO2 annual average)
    3. Supply of Green Areas
        (available Green Area per Person)
    4. Social Status
        (Social Urban Development Monitoring 2013, which relies on a set of six status and six dynamic indicators from the field of social reporting)
    5. Bioclimatic Stres
        (using the physiological equivalent temperature at night and day)
        
For more information visit: https://www.stadtentwicklung.berlin.de/umwelt/umweltatlas/ei901.htm
    
This data clearly shows how Environmental and Social issues are interlinked even on a city scale level. 

Future work may focus on linking environmental justice on different scales such as finding Trends resembled on City and Global scales.

I here show you how to explore and visualise this dataset. 



### Table of Contents

1. Berlin, Germany - A Case Study of measureing Enrivonmental Justice: 
    - Identifying KPIs on a city scale, which relate to the environmental and social issues that are discussed in the CDP survey data

## Berlin, Germany - A Case Study of measureing Enrivonmental Justice

![](https://static.ticketbar.eu/_1980x1320_/berlin/classificaties/rabattkarten/berlin-online-city-pass/berlincitypass_header-1516978149-1518186949.jpg)

In [ ]:
##imports

import numpy as np
import pandas as pd
import os
import geopandas as gpd

import geopandas as gpd
import pandas as pd
import shapely.geometry
from shapely.geometry import Polygon
import json

import plotly.graph_objects as go
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.express as px

In [ ]:
#load dataframes
df_berlin = pd.read_csv("../input/environmental-justice-in-berlin-city/BER_environmental_justice.csv")

berlin_neighbourhoods_df = gpd.read_file('../input/environmental-justice-in-berlin-city/LOR-Planungsraeume.geojson')
berlin_neighbourhoods_df = berlin_neighbourhoods_df.sort_values(by = "Name").reset_index(drop = True)

# Visualisation

### Social Status

Here you see the Social Status among all 447 Berlin planning areas.

You can hover, zoom in and out to get more detailed information on each neighbourhood. 

In [ ]:
# Data
cols_dd = ['Social Status']
value = ['Social Status']
# we need to add this to select which trace 
# is going to be visible
visible = np.array(cols_dd)

traces = []
buttons = []

    
g = gpd.GeoSeries(berlin_neighbourhoods_df.geometry.tolist())

poly_json = json.loads(g.to_json())

df = pd.DataFrame([np.arange(len(berlin_neighbourhoods_df)), df_berlin[value].values.flatten()]).T
df.columns = ["id", "color"]
        
labels = ['Very Low','Low','Middle', "High"]

traces.append(go.Choropleth(geojson=poly_json,
                            locations=df['id'].astype(str),
                            autocolorscale = False,
                            z = df['color'].astype(float),
                            zauto = False,
                            zmin = 1, 
                            zmax = 4,
                            colorbar=dict(
                                thicknessmode="pixels",
                                lenmode="fraction",
                                yanchor="top",y=1,
                                ticks="outside",
                                tickvals=[1,2,3,4],
                                ticktext=labels,
                                dtick=4),
                            showlegend = False,
                            colorscale=[[0, 'rgb(60,60,255)'],
                                        [1, 'rgb(255,60,60)']],))


fig = go.Figure()


fig.add_trace(traces[0])

fig.update_geos(fitbounds="locations", visible=False)


first_title = cols_dd[0]
fig.update_layout(title=f"<b>{first_title}</b>",title_x=0.5,
    autosize=True,
    )

fig.show()

### Bringing together environmental and social indicators

Here we plot the different environmental factors in a choropleth plot and plot PieCharts to visualise the environmental factors in neighbourhoods with different social statuses. 

We can clearly see that with higher social status a neighbourhood is less noisy, has more access to green areas, the air is less polluted and bioclimatic stress is reduced. 

Per example: Below you can see that nearly 50% of neighbourhoods with high social status have low / very low air pollution , while only < 15% of neighbourhoods with middle and low and very low social status have low / very low air pollution.



You can select the different environmental KPIs in the drop down menu at the top. 

In [ ]:
# Data
cols_dd = ['Air Pollution (PM2.5 + NO2)', 
        'Noise',
        'Grean area supply', 
        'Bioclimatic Stress', ]
# we need to add this to select which trace 
# is going to be visible
visible = np.array(cols_dd)

# define traces and buttons at once
traces = []
piecharts = []
buttons = []

#loop through environmental factors
for idx, value in enumerate(cols_dd):
    
    #define all relevant dataframes for the plot
    g = gpd.GeoSeries(berlin_neighbourhoods_df.geometry.tolist())

    poly_json = json.loads(g.to_json())

    df = pd.DataFrame([np.arange(len(berlin_neighbourhoods_df)), df_berlin[value]]).T
    df.columns = ["id", "color"]

            
    labels = ['Very Low','Low','Middle', "High"]
    
    #add the different choropleth maps
    traces.append(go.Choropleth(geojson=poly_json,
                                locations=df['id'].astype(str),
                                autocolorscale = False,
                                z = df['color'].astype(float),
                                zauto = False,
                                zmin = 1, 
                                zmax = 4,
                                colorbar=dict(
                                    thicknessmode="pixels",
                                    lenmode="fraction",
                                    yanchor="top",y=1, len=0.3,
                                    ticks="outside",
                                    tickvals=[1,2,3,4],
                                    ticktext=labels,
                                    dtick=4),
                                showlegend = False,
                                colorscale=[[0, 'rgb(60,60,255)'],
                                            [1, 'rgb(255,60,60)']],))
    
    #so that there is no visualisation error
    boolx = True
    
    if(idx) > 0:
        boolx = False

    #pie chart color
    berlin_colors = ['rgb(255,60,60)', 'rgb(180, 60, 255)', 'rgb(120, 60, 255)', 'rgb(60,60,255)']
    
    #create pie charts
    for i in range(1, 5):
        values = [
                    (df_berlin[df_berlin["Social Status"] == i][value] == 4).sum(), 
                    (df_berlin[df_berlin["Social Status"] == i][value] == 3).sum(), 
                    (df_berlin[df_berlin["Social Status"] == i][value] == 2).sum(), 
                    (df_berlin[df_berlin["Social Status"] == i][value] == 1).sum()]
        
        labels = ["High", 'Middle','Low', 'Very Low']

        piecharts.append(go.Pie(labels=labels, values=values, showlegend = True, visible = boolx, marker_colors=berlin_colors, sort = False))
    
    #create drop down
    buttons.append(dict(label=value.split(":")[0],
                        method="update",
                        args=[{"visible":list(visible==value)},
                              {"title":f"<b>{value}</b>"}]))

#drop down design
updatemenus = [{"active":0,
                "buttons":buttons,
                "direction":"down",
                "x":0.05,
                "xanchor":"left",
                "y":1.02,
                "yanchor":"top"
               }]


#make subplots 
fig = make_subplots(
    rows=4, cols=2,
    
    specs=[[{"rowspan": 2, "colspan": 2, 'type': 'choropleth'}, None],
           [None, None],
          [{'type': 'pie'}, {'type': 'pie'}],
          [{'type': 'pie'}, {'type': 'pie'}]],
    
    subplot_titles=("", "high Social Status", "medium Social Status", "low Social Status",  "very low Social Status"),
    
    print_grid=False)

## add traces to Figure in the correct order
for i in range(len(cols_dd)):
    fig.add_trace((traces[i]), 1, 1)
for i in range(len(cols_dd)):
    fig.add_trace((piecharts[4 * i + 3]), 3, 1)
for i in range(len(cols_dd)):
    fig.add_trace((piecharts[4 * i + 2]), 4, 1)
for i in range(len(cols_dd)):
    fig.add_trace((piecharts[4 * i + 1]), 3, 2)
for i in range(len(cols_dd)):
    fig.add_trace((piecharts[4 * i + 0]), 4, 2)
    
    
#zoom in from world to berlin
fig.update_geos(fitbounds="locations", visible=False)


#move subplot titles
fig.layout.annotations[0].update(x=0.075)
fig.layout.annotations[1].update(x=0.625)
fig.layout.annotations[2].update(x=0.025)
fig.layout.annotations[3].update(x=0.675)

#update layout button style, size and subplot legend to be at the bottom
first_title = cols_dd[0]
fig.update_layout(dict(updatemenus=updatemenus), title=f"<b>{first_title}</b>",title_x=0.5, width=800,
    height=900,
    autosize=False,
    legend=dict(
        x=1,
        y=0.5,
        traceorder='normal',
        font=dict(
            size=12,),
    ))

fig.show()

Work in Progress....